## In this file we convert a CSV into a pandas dataframe to join with the state borders json file.


In [ ]:
import pandas as pd
import csv
import json
from flask import Flask
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np

In [ ]:
stategeo_df = pd.read_csv("CleanData/state_geojson.csv")
stategeo_df.head()

In [13]:
state = ['Maine', 'Massachusetts', 'Michigan', 'Montana', 'Nevada', 'New Jersey', 'New York', 'North Carolina', 'Ohio', 'Pennsylvania', 'Rhode Island', 'Tennessee', 'Texas', 'Utah', 
'Washington', 'Wisconsin', 'Puerto Rico', 'Maryland', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida',
'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Lousiana', 'Minnesota', 'Mississippi', 'Missouri', 'Nebraska', 'New Hampshire', 
'New Mexico', 'North Dakota', 'Oklahoma', 'Oregon', 'South Carolina', 'South Dakota', 'Vermont', 'Virginia', 'West Virginia', 'Wyoming']

In [19]:
stategeo_df['state'] = state
stategeo_df.head()

,type,properties,geometry,NAME,state
0,Feature,"{'GEO_ID': '0400000US04', 'STATE': '04', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-112.538...",Maine,Maine
1,Feature,"{'GEO_ID': '0400000US05', 'STATE': '05', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-94.0429...",Massachusetts,Massachusetts
2,Feature,"{'GEO_ID': '0400000US06', 'STATE': '06', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-1...",Michigan,Michigan
3,Feature,"{'GEO_ID': '0400000US08', 'STATE': '08', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-107.317...",Montana,Montana
4,Feature,"{'GEO_ID': '0400000US09', 'STATE': '09', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-72.3974...",Nevada,Nevada


In [20]:
state_df = pd.read_csv("CleanData/state_data.csv", encoding="utf-8")
state_df.head()

,index,type,coordinates,state,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,...,Republican,Democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
0,0,MultiPolygon,"[[[[-67.619761, 44.519754], [-67.61541, 44.52...",Maine,786337,341949,420357,6711,147,1330158,...,False,True,19145.0,1279415.0,10504.0,10420.0,10504.0,169.0,504.526530,11.051319
1,1,MultiPolygon,"[[[[-70.832044, 41.606504], [-70.823735, 41.5...",Massachusetts,3482216,1131042,2271763,159707,10013,6789319,...,False,True,630365.0,5455488.0,399548.0,20027.0,282388.0,1502.0,2352.327236,147.481655
2,2,MultiPolygon,"[[[[-88.684434, 48.115785], [-88.675628, 48.1...",Michigan,5531329,2647004,2795649,197406,7699,9925568,...,False,True,351495.0,8949166.0,393596.0,128261.0,101336.0,1713.0,1988.863509,77.567349
3,3,Polygon,"[[[-104.057698, 44.997431], [-104.250145, 44....",Montana,602781,341765,243716,33495,376,1029862,...,True,False,30486.0,910095.0,2328.0,81715.0,4918.0,319.0,3252.377503,36.509746
4,4,Polygon,"[[[-114.0506, 37.000396], [-114.049995, 36.95...",Nevada,1337217,635017,671887,101477,1781,2887725,...,False,True,510752.0,2132180.0,64538.0,113809.0,59854.0,6593.0,3514.081154,61.674848


In [21]:
state_df['hispanic_pop'] = round(state_df['TotalPop']*state_df['Hispanic'])

state_df['white_pop'] = round(state_df['TotalPop']*state_df['White'])

state_df['black_pop'] = round(state_df['TotalPop']*state_df['Black'])

state_df['native_pop'] = round(state_df['TotalPop']*state_df['Native'])

state_df['asian_pop'] = round(state_df['TotalPop']*state_df['Asian'])

state_df['pacific_pop'] = round(state_df['TotalPop']*state_df['Pacific'])

In [22]:
merged_df = pd.merge(stategeo_df, state_df, on="state")
merged_df.head()
clean_state_df = merged_df.drop(['state'],axis=1)

clean_state_df["cases_per_100k"]= (clean_state_df["cases"]/clean_state_df["TotalPop"])*100000
clean_state_df["deaths_per_100k"]= (clean_state_df["deaths"]/clean_state_df["TotalPop"])*100000


clean_state_df.head()

,type_x,properties,geometry,NAME,index,type_y,coordinates,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,...,Republican,Democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
0,Feature,"{'GEO_ID': '0400000US04', 'STATE': '04', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-112.538...",Maine,0,MultiPolygon,"[[[[-67.619761, 44.519754], [-67.61541, 44.52...",786337,341949,420357,...,False,True,19145.0,1279415.0,10504.0,10420.0,10504.0,169.0,504.526530,11.051319
1,Feature,"{'GEO_ID': '0400000US05', 'STATE': '05', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-94.0429...",Massachusetts,1,MultiPolygon,"[[[[-70.832044, 41.606504], [-70.823735, 41.5...",3482216,1131042,2271763,...,False,True,630365.0,5455488.0,399548.0,20027.0,282388.0,1502.0,2352.327236,147.481655
2,Feature,"{'GEO_ID': '0400000US06', 'STATE': '06', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-1...",Michigan,2,MultiPolygon,"[[[[-88.684434, 48.115785], [-88.675628, 48.1...",5531329,2647004,2795649,...,False,True,351495.0,8949166.0,393596.0,128261.0,101336.0,1713.0,1988.863509,77.567349
3,Feature,"{'GEO_ID': '0400000US08', 'STATE': '08', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-107.317...",Montana,3,Polygon,"[[[-104.057698, 44.997431], [-104.250145, 44....",602781,341765,243716,...,True,False,30486.0,910095.0,2328.0,81715.0,4918.0,319.0,3252.377503,36.509746
4,Feature,"{'GEO_ID': '0400000US09', 'STATE': '09', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-72.3974...",Nevada,4,Polygon,"[[[-114.0506, 37.000396], [-114.049995, 36.95...",1337217,635017,671887,...,False,True,510752.0,2132180.0,64538.0,113809.0,59854.0,6593.0,3514.081154,61.674848


In [24]:
# #new_df = pd.DataFrame(clean_state_df.geometry.str.split(',',1).tolist(), 
#                          #columns = ['type','coordinates'])

clean_state_df['geometry'] = clean_state_df['geometry'].str.replace("'",'"') 


clean_state_df.head()

,type_x,properties,geometry,NAME,index,type_y,coordinates,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,...,Republican,Democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
0,Feature,"{'GEO_ID': '0400000US04', 'STATE': '04', 'NAME...","{""type"": ""Polygon"", ""coordinates"": [[[-112.538...",Maine,0,MultiPolygon,"[[[[-67.619761, 44.519754], [-67.61541, 44.52...",786337,341949,420357,...,False,True,19145.0,1279415.0,10504.0,10420.0,10504.0,169.0,504.526530,11.051319
1,Feature,"{'GEO_ID': '0400000US05', 'STATE': '05', 'NAME...","{""type"": ""Polygon"", ""coordinates"": [[[-94.0429...",Massachusetts,1,MultiPolygon,"[[[[-70.832044, 41.606504], [-70.823735, 41.5...",3482216,1131042,2271763,...,False,True,630365.0,5455488.0,399548.0,20027.0,282388.0,1502.0,2352.327236,147.481655
2,Feature,"{'GEO_ID': '0400000US06', 'STATE': '06', 'NAME...","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-1...",Michigan,2,MultiPolygon,"[[[[-88.684434, 48.115785], [-88.675628, 48.1...",5531329,2647004,2795649,...,False,True,351495.0,8949166.0,393596.0,128261.0,101336.0,1713.0,1988.863509,77.567349
3,Feature,"{'GEO_ID': '0400000US08', 'STATE': '08', 'NAME...","{""type"": ""Polygon"", ""coordinates"": [[[-107.317...",Montana,3,Polygon,"[[[-104.057698, 44.997431], [-104.250145, 44....",602781,341765,243716,...,True,False,30486.0,910095.0,2328.0,81715.0,4918.0,319.0,3252.377503,36.509746
4,Feature,"{'GEO_ID': '0400000US09', 'STATE': '09', 'NAME...","{""type"": ""Polygon"", ""coordinates"": [[[-72.3974...",Nevada,4,Polygon,"[[[-114.0506, 37.000396], [-114.049995, 36.95...",1337217,635017,671887,...,False,True,510752.0,2132180.0,64538.0,113809.0,59854.0,6593.0,3514.081154,61.674848


In [25]:
clean_state_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type_x                49 non-null     object 
 1   properties            49 non-null     object 
 2   geometry              49 non-null     object 
 3   NAME                  49 non-null     object 
 4   index                 49 non-null     int64  
 5   type_y                49 non-null     object 
 6   coordinates           49 non-null     object 
 7   total_votes20         49 non-null     int64  
 8   votes20_Donald_Trump  49 non-null     int64  
 9   votes20_Joe_Biden     49 non-null     int64  
 10  cases                 49 non-null     int64  
 11  deaths                49 non-null     int64  
 12  TotalPop              49 non-null     int64  
 13  VotingAgeCitizen      49 non-null     int64  
 14  Men                   49 non-null     float64
 15  Women                 49 

In [26]:
clean_state_df.corr()

,index,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,VotingAgeCitizen,Men,Women,...,Republican,Democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
index,1.000000,-0.288613,-0.286157,-0.278442,-0.242779,-0.374099,-0.284522,-0.297740,0.224585,-0.224585,...,0.239600,-0.239600,-0.159082,-0.329276,-0.118163,-0.080307,-0.156633,-0.003411,0.062451,-0.360859
total_votes20,-0.288613,1.000000,0.966664,0.982389,0.941603,0.759464,0.981448,0.987391,-0.249984,0.249984,...,-0.149712,0.149712,0.810858,0.965251,0.608175,0.345585,0.703691,0.197395,-0.010817,0.237799
votes20_Donald_Trump,-0.286157,0.966664,1.000000,0.901836,0.942834,0.739064,0.933877,0.948318,-0.275198,0.275198,...,-0.012621,0.012621,0.733097,0.937930,0.662949,0.265556,0.528593,0.113060,0.056183,0.221879
votes20_Joe_Biden,-0.278442,0.982389,0.901836,1.000000,0.901351,0.744509,0.974829,0.974495,-0.223218,0.223218,...,-0.245466,0.245466,0.832205,0.944658,0.545649,0.388545,0.801746,0.250666,-0.059576,0.242714
cases,-0.242779,0.941603,0.942834,0.901351,1.000000,0.793120,0.950714,0.946885,-0.227123,0.227123,...,-0.003249,0.003249,0.834104,0.906067,0.664208,0.331814,0.624496,0.168757,0.178125,0.283572
deaths,-0.374099,0.759464,0.739064,0.744509,0.793120,1.000000,0.808382,0.817778,-0.348014,0.348014,...,-0.206706,0.206706,0.575564,0.838096,0.553393,0.173056,0.529614,0.068228,0.026719,0.645851
TotalPop,-0.284522,0.981448,0.933877,0.974829,0.950714,0.808382,1.000000,0.997260,-0.230819,0.230819,...,-0.133564,0.133564,0.866354,0.967824,0.572507,0.368787,0.761834,0.229636,-0.003079,0.266507
VotingAgeCitizen,-0.297740,0.987391,0.948318,0.974495,0.946885,0.817778,0.997260,1.000000,-0.261010,0.261010,...,-0.134969,0.134969,0.830279,0.980472,0.590772,0.347708,0.735453,0.209309,-0.009088,0.280794
Men,0.224585,-0.249984,-0.275198,-0.223218,-0.227123,-0.348014,-0.230819,-0.261010,1.000000,-1.000000,...,0.237336,-0.237336,0.008136,-0.287950,-0.477819,0.222303,-0.042693,0.162606,0.128415,-0.529177
Women,-0.224585,0.249984,0.275198,0.223218,0.227123,0.348014,0.230819,0.261010,-1.000000,1.000000,...,-0.237336,0.237336,-0.008136,0.287950,0.477819,-0.222303,0.042693,-0.162606,-0.128415,0.529177


In [53]:
def df_to_geojson(clean_state_df):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in clean_state_df.iterrows():

        geometry = json.loads(row['geometry'])

        feature = { 'type':'Feature',
                    'geometry': geometry,
                   'properties': {"state": row['NAME'], "total_votes20": row['total_votes20'], "votes20_Donald_Trump": row['votes20_Donald_Trump'], "votes20_Joe_Biden": row['votes20_Joe_Biden'], "cases": row['cases'], "deaths": row['deaths'], "TotalPop": row['TotalPop'], 'VotingAgeCitizen': row['VotingAgeCitizen'], "Men": row['Men %'],
    "Women": row['Women %'], 'Hispanic': row['Hispanic %'], 'White': row['White %'], 'Black': row['Black %'], 'Native': row['Native %'], 'Asian': row['Asian %'], 'Pacific': row['Pacific %'], 'Biden': row['Biden %'], 'Trump': row['Trump %'], 'Republican': row['Republican'], 'Democrat': row['Democrat'], 'hispanic_pop': row['hispanic_pop'], 'white_pop': row['white_pop'], 'black_pop': row['black_pop'], 'native_pop': row['native_pop'], 'asian_pop': row['asian_pop'], 'pacific_pop': row['pacific_pop'], 'cases_per_100k': row['cases_per_100k'], 'deaths_per_100k': row['deaths_per_100k']}
        }
        #print({ "geometry": {row.geometry['coordinates']}} )
        #feature['geometry']['coordinates'] = [row[lon],row[lat]]
        #for prop in properties:
            #feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)

    return geojson

In [56]:
## Create PostgreSQL RDS Database Connection
# conn_string = "postgres:{pwd}@project2.cy7mfzyzcf0j.us-east-2.rds.amazonaws.com:5432/project2db"
# engine = create_engine(f'postgresql://{conn_string}')
# Check table names
# engine.table_names()
# import csv
# with open('state_data.csv') as csvfile:
#     myCSVReader = csv.DictReader(csvfile)
#     change names in placeholder to match names in csv file.
#     sql = """INSERT INTO State(index,type,coordinates,state,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,VotingAgeCitizen,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Biden,Trump,Republican,Democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
# )
#           VALUES (%(index)s,%(type)s,%(coordinates)s,%(state)s,%(total_votes20)s,%(votes20_Donald_Trump)s,%(votes20_Joe_Biden)s,%(cases)s,%(deaths)s,%(TotalPop)s, %(VotingAgeCitizen)s,%(Men)s,%(Women)s, %(Hispanic)s,%(White)s,%(Black)s, %(Native)s,%(Asian)s,%(Pacific)s, %(Biden)s,%(Trump)s,%(Republican)s, %(Democrat)s,%(hispanic_pop)s,%(white_pop)s, %(black_pop)s,%(native_pop)s,%(asian_pop)s, %(pacific_pop)s,%(cases_per_100k)s,%(deaths_per_100k)s)"""
#     for row in myCSVReader:
#         use row directly
#         engine.execute(sql, row)
#pd.read_sql_query('select * from state', con=engine).head()

In [57]:
#final_df.to_csv('state_data.csv')